# ロジスティック回帰


sklearn.linear_model.LogisticRegression
- 引数
    - penalty: 'l1', 'l2', 'elasticnet', 'none'
    - solver: 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'
    - multi_class: 'ovr', 'auto', 'multinomial'
- .predict(X)でラベル(クラス)の分類結果を取得
- .predict_proba(X)でラベル(クラス)の確率p(X)を取得

sklearn.metrics.log_loss
- log_loss(y_true, y_pred_proba)
    - y_predには.predict_proba(X)の戻り値を入れる

## データセット準備

In [2]:
import seaborn as sns
df = sns.load_dataset('titanic').dropna()
df.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,First,woman,False,C,Southampton,yes,True


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
X = df.loc[:, (df.columns!='survived') & (df.columns!='alive')]
X = pd.get_dummies(X, drop_first=True)
y = df['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


## ロジスティック回帰

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

/Users/hiroto/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


logloss, coefficient, intercept, classes, features namesなど

In [23]:
from sklearn.metrics import log_loss
log_loss(y_test, y_pred_proba), model.coef_, model.intercept_,\
model.classes_, model.feature_names_in_

(0.4111567729103682,
 array([[-0.29613445, -0.02130647,  0.62287823, -0.37405926,  0.00478665,
         -0.88713974,  0.17503682,  0.38845621, -0.4045677 , -0.02303382,
         -0.4531837 , -0.40475737, -0.88713974,  1.21198449, -0.11127389,
         -1.20757958, -0.14162972,  0.61110768, -0.13071674, -0.5516418 ,
         -0.4045677 , -0.02303382]]),
 array([2.14323822]),
 array([0, 1]),
 array(['pclass', 'age', 'sibsp', 'parch', 'fare', 'adult_male', 'alone',
        'sex_male', 'embarked_Q', 'embarked_S', 'class_Second',
        'class_Third', 'who_man', 'who_woman', 'deck_B', 'deck_C',
        'deck_D', 'deck_E', 'deck_F', 'deck_G', 'embark_town_Queenstown',
        'embark_town_Southampton'], dtype=object))